In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

import scipy.stats as scs
from scipy.special import comb
from statsmodels.stats.proportion import proportions_ztest

import how_to_ab_test.sample_data as sample_data

# How to AB Test

This lab explores how one goes about AB testing.

# Tests

### Fisher Exact Test


The Fisher exact test looks at the probability of this or more extreme a skew of success rates in a randomly drawn 2x2 contingency table conditioned on having the same marginals.

Let
<ul> 
    <li style="list-style-type: none;">$N$ &nbsp;&nbsp; be the total number of draws;</li>
    <li style="list-style-type: none;">$n$ &nbsp;&nbsp; the number of draws in Group 0;</li>
    <li style="list-style-type: none;">$K$ &nbsp;&nbsp; the total number of successes;</li>
</ul>

Then, conditioning on the marginals leaves a single degree of freedom. This can, WLOG, be taken to be
<ul>
    <li style="list-style-type: none;">$k$ &nbsp;&nbsp; the number of successes in Group 0;</li>
</ul>

Then, more extreme cases are $k > k_0$. (With the corresponding other $k < K-k_0$ for the two-sided test.)

For the probability of this table being drawn ($k = k$) given the marginals, consider the analogy to drawing without replacement $k$ balls of type 1 and $n-k$ of type 2 from an urn with $K$ of type 1 and $N-K$ of type 2.

To calculate this, we want the number of ways to draw the first row with the given split ($k$ out of $K$ from type 1 and $n-k$ from $N-K$) out of the possible ways to draw the first row ($n$ from a total $N$). (Note - having picked the first row, that fixes the whole table.)

So we get the hypergeometric distribution:
$$
P(t_{0,0} = k) = \frac{{K \choose k}{N-K \choose n-k}}{{N \choose n}}
$$

For the p-value of the Fisher exact test, we then just sum over values of $k$ that are as or more extreme than the observed one.

Implemented in `scipy.stats.fisher_exact`:

In [3]:
def fisher_exact(table):
    N = table.sum().sum()
    n = table.sum(axis=1)[0]
    K = table.sum(axis=0)[0]
    k = table.iloc[0, 0]

    num = 0
    for ki in range(k, K+1):
        num += comb(K, ki, exact=True) * comb(N-K, n-ki, exact=True)

    return num / comb(N, n, exact=True)


for table in sample_data.random_contingency_tables_2x2(10, is_equal_p=True):
    odds_ratio, pvalue = scs.fisher_exact(table, alternative='greater')
    pvalue_ = fisher_exact(table)
    # print((pvalue, pvalue_))
    assert abs(pvalue-pvalue_)<1e-8

Follow Up: It looks very Bayes-like. Is there any interpretation along those lines?

### Large Sample Z Test

The z test is a large-sample approximation that the proportions in two groups are equal.

It uses the fact that, by the CLT (and in practice converging very quickly - see below), the natural estimator for the group proportion is roughly normal:

$$
\hat{p} := \frac{X}{n} \sim \mathcal{N}\Big(p, \frac{p(1-p)}{n}\Big)
$$

where $ X \sim Binom(p, n)$.

Note - It is a normal distribution, not a t. I haven't found a reference for this. Beyond that we are talking about large-sample (and so the t-test should be approximately normal anyways), I suspect that we have a normal distribution because the binomial is a single-parameter distribution so the extra degree of freedom for the SD doesn't exist in this case.

To test the null hypothesis that the two groups come from the same distribution, we consider:

$$
z := \hat{p_1} - \hat{p_2} \sim \mathcal{N}\big(0, \hat{s}^2 \big)
$$

where 

$$
\begin{align}
\hat{s}^2 &= Var(\hat{p}) \\
    &= Var(\hat{p_1}) + Var(\hat{p_2}) \\
    &= \frac{\hat{p_1}(1-\hat{p_1})}{n_1} + \frac{\hat{p_2}(1-\hat{p_2})}{n_2}
\end{align}
$$

called the Satterthwaite approximation or we can pool, under the null hypothesis, the groups when estimating the SD, giving the pooled approximation

$$
\begin{align}
\hat{s}^2 &= Var(\hat{p}) \\
    &= Var(\hat{p_1}) + Var(\hat{p_2}) \\
    &= \frac{\hat{p}(1-\hat{p})}{n_1} + \frac{\hat{p}(1-\hat{p})}{n_2} \\
    &= \hat{p}(1-\hat{p}) \Big( \frac{1}{n_1} + \frac{1}{n_2} \Big)
\end{align}
$$

where

$$
\hat{p} = \frac{n_1 p_1 + n_2 p_2}{n_1 + n_2} = \frac{x_1 + x_2}{n_1 + n_2}
$$

The usual one-sided or two-sided tests are then as usual. 

Implemented in `statsmodels.stats.proportion.proportions_ztest`:

In [4]:
def proportions_ztest_(table, *, pooled_approx=False):
    assert table.shape == (2, 2)

    s1, s2 = table.iloc[:, 0]
    n1, n2 = table.sum(axis=1)

    p1, p2 = s1 / n1, s2 / n2

    if pooled_approx:
        p = (s1 + s2) / (n1 + n2)
        v = p * (1 - p) * (1 / n1 + 1 / n2)
    else:
        v = p1 * (1 - p1) / n1 + p2 * (1 - p2) / n2

    if not v > 0:
        return np.nan, np.nan
    
    z_stat = (p1 - p2) / np.sqrt(v)

    return z_stat, scs.norm.sf(np.abs(z_stat)) * 2


for table in sample_data.random_contingency_tables_2x2(10, is_equal_p=True):
    z_stat, pvalue = proportions_ztest(
        table.Success, 
        table.Success+table.Fail, 
        alternative='two-sided',
        prop_var=False
    )
    z_stat_, pvalue_ = proportions_ztest_(table, pooled_approx=True) 
    # print((pvalue, pvalue_))
    assert (np.isnan(pvalue) and np.isnan(pvalue_)) or abs(pvalue-pvalue_)<1e-8

### Chi-Squared Test

This is a large-sample approximation that two multinominial distribution match. Note that, unlike the Fisher and Z tests, it is limited to 2-sided tests but applies for any number of categories and groups.

Given a multinomial sample 

$$
\begin{align}
    X &\sim Multinomial_k(n, \pi) \\
    P(X=x) &= \frac{n!}{x_1!...x_k!} \pi_1^{x_1} ... \pi_k^{x_k}
\end{align}
$$

it is shown that under $ H_0: \pi = \pi_0 $ we have

$$
X^2 = \sum_{i=1}^k \frac{(X_i - E_i)^2}{E_i} \sim \chi^2(k-1)
$$

where $ E_i = X_i * \pi_i $.

Extending to $g$ groups, we have

$$
X^2 = \sum_{cells} \frac{(X_i - E_i)^2}{E_i} = \sum_{i=1}^{k} \sum_{j=1}^g \frac{(X_i^{(g)}-E_i)^2}{E_i} \sim \chi^2\big((k-1)(g-1)\big)
$$

where $E_i$ is estimated by the category totals $ E_i = \sum_{j=1}^g X_i^{(g)}$

Note: Cressie & Read extended this by remarking that this and a few other tests were related as special cases of a continuous family of the "power divergence" statistics:

$$
2n I^\lambda = \frac{2}{\lambda (\lambda + 1)} \sum_{i=1}^k X_i\Bigg( \Big(\frac{X_i}{E_i}\Big)^\lambda - 1 \Bigg)
$$

Implemented in `scipy.stats.chi2_contingency`:

In [5]:
def chi2_contingency(table):
    by_category = table.values.sum(axis=0, keepdims=True)
    pi = by_category/by_category.sum()
    by_group = table.values.sum(axis=1, keepdims=True)

    E = by_group*pi
    
    X2 = (np.square(table.values-E)/E).sum()
    
    dof = (table.shape[0]-1)*(table.shape[1]-1)
    
    return X2, 1.0 - scs.chi2.cdf(X2, dof), dof, E


n = 10
n_valid = 0
for table in sample_data.random_contingency_tables_2x2(n, is_equal_p=True):
    if not (table.sum(axis=0) == 0).any():
        # chi2_contingency invalid if any category has 0 observed
        n_valid += 1
        chi2, pvalue, dof, expected = scs.chi2_contingency(table.values, correction=False)
        chi2_, pvalue_, dof_, expected_ = chi2_contingency(table)
        # print((pvalue, pvalue_))
        assert abs(pvalue-pvalue_)<1e-8
# print((n_valid, n))       
assert n_valid > n/2

# References

### Tests

https://www.itl.nist.gov/div898/handbook/prc/section3/prc33.htm

https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/PASS/Tests_for_Two_Proportions.pdf

https://www.jstor.org/stable/2983604

https://towardsdatascience.com/the-math-behind-a-b-testing-with-example-code-part-1-of-2-7be752e1d06f

https://www.jstor.org/stable/2345686

### Methodology

http://blog.analytics-toolkit.com/2017/statistical-significance-ab-testing-complete-guide/

https://www.analytics-toolkit.com/pdf/Efficient_AB_Testing_in_Conversion_Rate_Optimization_-_The_AGILE_Statistical_Method_2017.pdf

https://www.evanmiller.org/how-not-to-run-an-ab-test.html

https://www.chrisstucchio.com/blog/2015/dont_use_bandits.html